In [52]:
import sys  
sys.path.insert(0, './../')
import numpy as np
import bbdc2021
import pandas as pd
from importlib import reload
reload(bbdc2021)
import tensorflow as tf
from keras import layers
import keras

In [39]:
X_train_orig, Y_train_orig = bbdc2021.read_train("./../data/dataset_fft")
X_train_cropped = [np.delete(x,1,0) for x in X_train_orig]
Y_train_cropped = [np.delete(x,1,0) for x in Y_train_orig]
X_train, Y_train, X_validation, Y_validation, X_test, Y_test = bbdc2021.splitTrain(X_train_cropped, Y_train_cropped)

In [80]:
channels = [32, 64, 128]
inputShape = X_train[0].shape
print(inputShape)

#input_layer = layers.Input(shape=(296,296,1))
input_layer = layers.Input(shape=(inputShape))
##############################
x = input_layer
# Encoder Start
x = layers.Conv1D(channels[0], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Conv1D(channels[0], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
sh1 = layers.ReLU()(x)

x = layers.MaxPooling1D()(sh1)
#############################
x = layers.Conv1D(channels[1], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Conv1D(channels[1], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
sh2 = layers.ReLU()(x)
x = layers.MaxPooling1D()(sh2)
#############################
x = layers.Conv1D(channels[2], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Conv1D(channels[2], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
#Decoder start
x = layers.UpSampling1D(2)(x)

############################
#sh2 = layers.Cropping1D(cropping=10)(sh2)
x = layers.Concatenate()([x,sh2])
x = layers.Conv1D(channels[1], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Conv1D(channels[1], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.UpSampling1D(2)(x)
#############################
#sh1 = layers.Cropping1D(cropping=16)(sh1)
x = layers.Concatenate()([x,sh1])
x = layers.Conv1D(channels[0], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Conv1D(channels[0], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Conv1D(13, kernel_size=1, activation="softmax", padding='same')(x)

model = keras.models.Model(inputs=input_layer, outputs=x)
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['mae'])

(208, 16)
Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           (None, 208, 16)      0                                            
__________________________________________________________________________________________________
conv1d_149 (Conv1D)             (None, 208, 32)      1568        input_21[0][0]                   
__________________________________________________________________________________________________
batch_normalization_140 (BatchN (None, 208, 32)      128         conv1d_149[0][0]                 
__________________________________________________________________________________________________
re_lu_139 (ReLU)                (None, 208, 32)      0           batch_normalization_140[0][0]    
_________________________________________________________________________________

In [81]:
history = model.fit(np.array(X_train),np.array(Y_train), batch_size=15, epochs=15, validation_data=(np.array(X_validation), np.array(Y_validation)))

Train on 8000 samples, validate on 1000 samples
Epoch 1/15
8000/8000 [==============================] - 92s 11ms/step - loss: 0.1224 - mae: 0.0677 - val_loss: 0.1088 - val_mae: 0.0594
Epoch 2/15
8000/8000 [==============================] - 67s 8ms/step - loss: 0.0845 - mae: 0.0454 - val_loss: 0.1904 - val_mae: 0.0783
Epoch 3/15
8000/8000 [==============================] - 56s 7ms/step - loss: 0.0713 - mae: 0.0382 - val_loss: 0.1025 - val_mae: 0.0484
Epoch 4/15
8000/8000 [==============================] - 57s 7ms/step - loss: 0.0631 - mae: 0.0337 - val_loss: 0.1407 - val_mae: 0.0665
Epoch 5/15
8000/8000 [==============================] - 59s 7ms/step - loss: 0.0573 - mae: 0.0306 - val_loss: 0.1225 - val_mae: 0.0602
Epoch 6/15
 780/8000 [=>............................] - ETA: 50s - loss: 0.0548 - mae: 0.0296

KeyboardInterrupt: 